In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', 100)

import re
import string
import nltk

from textblob import TextBlob, Word
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
# found a major class imbalance after doing work in merge_and_chop, so I'm going to fix that here
# also going to create a csv of roughly 20k instead of doing the train, test, and calidate csv files due to vocabulary problems

#this df is made from merge_and_chop using the dataset files at https://www.kaggle.com/forgemaster/steam-reviews-dataset?select=reviews-1-115.csv
#none of these files are stored in github
df = pd.read_csv("../../../../trimmed/combined_split_reviews.csv")
df.head(3)

,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198219450288,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",a vac server baned me for being good .....nice why did this happen they writed me to purchase th...,True,True
1,76561198371171813,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...",only game i play,True,True
2,76561198170681091,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', ...","great game,play it every day,over 500 hours,would reccomend",True,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220605 entries, 0 to 3220604
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   steamid    int64 
 1   appid      int64 
 2   app_title  object
 3   app_tags   object
 4   review     object
 5   fps        bool  
 6   voted_up   bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 129.0+ MB


In [4]:
df = df.drop_duplicates(subset="review")
df = df.dropna()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 680220 entries, 0 to 821202
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   steamid    680220 non-null  int64 
 1   appid      680220 non-null  int64 
 2   app_title  680220 non-null  object
 3   app_tags   680220 non-null  object
 4   review     680220 non-null  object
 5   fps        680220 non-null  bool  
 6   voted_up   680220 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 32.4+ MB


In [6]:
shooters = df["app_title"].unique()
shooters = list(shooters)
len(shooters)

175

In [7]:
df["voted_up"].value_counts()

True     608134
False     72086
Name: voted_up, dtype: int64

In [8]:
df["voted_up"].value_counts(normalize=True)

True     0.894025
False    0.105975
Name: voted_up, dtype: float64

In [10]:
df["app_title"].value_counts()[:15]

Counter-Strike: Global Offensive            289088
Garry's Mod                                  66364
Left 4 Dead 2                                41017
Portal 2                                     35231
Borderlands 2                                21301
Fallout: New Vegas                           19197
Arma 3                                       17259
Portal                                       15165
Counter-Strike: Source                       11068
BioShock Infinite                            10084
Half-Life                                     9963
Counter-Strike                                8646
Star Wars: Battlefront 2 (Classic, 2005)      7069
Killing Floor                                 6571
Half-Life 2                                   5831
Name: app_title, dtype: int64

In [11]:
index_names = df[df["app_title"] == "Counter-Strike"].index
df.drop(index_names, inplace=True)

drop_games = ["Counter-Strike: Source", "Half-Life: Source", "Team Fortress Classic", "Day of Defeat: Source",
             "Half-Life Deathmatch: Source"]

for each in drop_games:
    index_names = df[df["app_title"] == each].index
    df.drop(index_names, inplace=True)
    
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 654899 entries, 0 to 821202
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   steamid    654899 non-null  int64 
 1   appid      654899 non-null  int64 
 2   app_title  654899 non-null  object
 3   app_tags   654899 non-null  object
 4   review     654899 non-null  object
 5   fps        654899 non-null  bool  
 6   voted_up   654899 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 31.2+ MB


In [17]:
df["app_title"].value_counts()[:15]

Counter-Strike: Global Offensive            289088
Garry's Mod                                  66364
Left 4 Dead 2                                41017
Portal 2                                     35231
Borderlands 2                                21301
Fallout: New Vegas                           19197
Arma 3                                       17259
Portal                                       15165
BioShock Infinite                            10084
Half-Life                                     9963
Star Wars: Battlefront 2 (Classic, 2005)      7069
Killing Floor                                 6571
Half-Life 2                                   5831
Call of Duty: World at War                    5483
Wolfenstein: The New Order                    4737
Name: app_title, dtype: int64

In [18]:
shooters = df["app_title"].unique()
shooters = list(shooters)
len(shooters)

169

In [19]:
df["voted_up"].value_counts()

True     584908
False     69991
Name: voted_up, dtype: int64

In [20]:
df["app_title"].value_counts(normalize=True)

Counter-Strike: Global Offensive    0.441424
Garry's Mod                         0.101335
Left 4 Dead 2                       0.062631
Portal 2                            0.053796
Borderlands 2                       0.032526
                                      ...   
Tribes: Ascend                      0.000018
Fallen Earth Free2Play              0.000012
Empires Mod                         0.000011
EverQuest II                        0.000009
Zombie Panic! Source                0.000006
Name: app_title, Length: 169, dtype: float64

In [21]:
df_csgo = df[df["app_title"] == "Counter-Strike: Global Offensive"]

In [24]:
index_names = df[df["app_title"] == "Counter-Strike: Global Offensive"].index
df.drop(index_names, inplace=True)

df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365811 entries, 0 to 365810
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   steamid    365811 non-null  int64 
 1   appid      365811 non-null  int64 
 2   app_title  365811 non-null  object
 3   app_tags   365811 non-null  object
 4   review     365811 non-null  object
 5   fps        365811 non-null  bool  
 6   voted_up   365811 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 14.7+ MB


In [25]:
df["voted_up"].value_counts(normalize=True)

True     0.931968
False    0.068032
Name: voted_up, dtype: float64

In [26]:
df_up = df[df["voted_up"] == 1]
df_down = df[df["voted_up"] == 0]

display(df_up.info(), df_down.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340924 entries, 0 to 365810
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   steamid    340924 non-null  int64 
 1   appid      340924 non-null  int64 
 2   app_title  340924 non-null  object
 3   app_tags   340924 non-null  object
 4   review     340924 non-null  object
 5   fps        340924 non-null  bool  
 6   voted_up   340924 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 16.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 24887 entries, 28 to 365775
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    24887 non-null  int64 
 1   appid      24887 non-null  int64 
 2   app_title  24887 non-null  object
 3   app_tags   24887 non-null  object
 4   review     24887 non-null  object
 5   fps        24887 non-null  bool  
 6   voted_up   24887 non-null  bool  
dtypes: b

None

None

In [31]:
X_up = df_up.drop("voted_up", axis=1)
y_up = df_up["voted_up"]

X_train, X_test, y_train, y_test = train_test_split(X_up, y_up, random_state=57, test_size=0.03)

In [32]:
up = pd.concat([X_test, y_test], axis=1)
up.reset_index(drop=True, inplace=True)
up.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10228 entries, 0 to 10227
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    10228 non-null  int64 
 1   appid      10228 non-null  int64 
 2   app_title  10228 non-null  object
 3   app_tags   10228 non-null  object
 4   review     10228 non-null  object
 5   fps        10228 non-null  bool  
 6   voted_up   10228 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 419.6+ KB


In [37]:
X_down = df_down.drop("voted_up", axis=1)
y_down = df_down["voted_up"]

X_train, X_test, y_train, y_test = train_test_split(X_down, y_down, random_state=57, test_size=0.4)

In [38]:
down = pd.concat([X_test, y_test], axis=1)
down.reset_index(drop=True, inplace=True)
down.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9955 entries, 0 to 9954
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    9955 non-null   int64 
 1   appid      9955 non-null   int64 
 2   app_title  9955 non-null   object
 3   app_tags   9955 non-null   object
 4   review     9955 non-null   object
 5   fps        9955 non-null   bool  
 6   voted_up   9955 non-null   bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 408.4+ KB


In [39]:
df_temp = pd.concat([up, down])
df_temp.reset_index(drop=True, inplace=True)
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20183 entries, 0 to 20182
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    20183 non-null  int64 
 1   appid      20183 non-null  int64 
 2   app_title  20183 non-null  object
 3   app_tags   20183 non-null  object
 4   review     20183 non-null  object
 5   fps        20183 non-null  bool  
 6   voted_up   20183 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 827.9+ KB


In [40]:
df_temp.head()

,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198271088129,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable', 'Building', 'Comedy', 'Co-op', 'Mod', 'First-Per...",good models\n,True,True
1,76561198138206834,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable', 'Building', 'Comedy', 'Co-op', 'Mod', 'First-Per...",I completely suck at making anything on here and death run keeps ending in countless deaths beca...,True,True
2,76561198128760839,70,Half-Life,"['FPS', 'Sci-fi', 'Action', 'Singleplayer', ""1990's"", 'Shooter', 'Multiplayer', 'First-Person', ...",There is not much to say about this old game that hasn't been said really. Many people have play...,True,True
3,76561198079636858,6060,"Star Wars: Battlefront 2 (Classic, 2005)","['Action', 'Multiplayer', 'Shooter', 'Third-Person Shooter', 'Classic', 'Sci-fi', 'Space', 'FPS'...",360 noscoped almost everything 10/10 even the single player is fun...,True,True
4,76561198238100200,220,Half-Life 2,"['FPS', 'Action', 'Sci-fi', 'Classic', 'Singleplayer', 'Story Rich', 'Shooter', 'First-Person', ...",No need for a review. It's practically history.,True,True


In [42]:
df_temp["app_title"].value_counts()

Garry's Mod                         2988
Left 4 Dead 2                       1710
Arma 3                              1176
Portal 2                            1154
Borderlands 2                        966
                                    ... 
EverQuest II                           2
Hamilton's Great Adventure             2
Pirates, Vikings, and Knights II       1
Penguins Arena: Sedna's World          1
Empires Mod                            1
Name: app_title, Length: 165, dtype: int64

In [43]:
#df_csgo

df_up = df_csgo[df_csgo["voted_up"] == 1]
df_down = df_csgo[df_csgo["voted_up"] == 0]

X_up = df_up.drop("voted_up", axis=1)
y_up = df_up["voted_up"]

X_train, X_test, y_train, y_test = train_test_split(X_up, y_up, random_state=57, train_size=1500)
up = pd.concat([X_train, y_train], axis=1)
up.reset_index(drop=True, inplace=True)

X_down = df_down.drop("voted_up", axis=1)
y_down = df_down["voted_up"]

X_train, X_test, y_train, y_test = train_test_split(X_down, y_down, random_state=57, train_size=1500)
down = pd.concat([X_train, y_train], axis=1)
down.reset_index(drop=True, inplace=True)


csgo = pd.concat([up, down])
csgo.reset_index(drop=True, inplace=True)
csgo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    3000 non-null   int64 
 1   appid      3000 non-null   int64 
 2   app_title  3000 non-null   object
 3   app_tags   3000 non-null   object
 4   review     3000 non-null   object
 5   fps        3000 non-null   bool  
 6   voted_up   3000 non-null   bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 123.2+ KB


In [44]:
final_df = pd.concat([df_temp, csgo])
final_df.reset_index(drop=True, inplace=True)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23183 entries, 0 to 23182
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    23183 non-null  int64 
 1   appid      23183 non-null  int64 
 2   app_title  23183 non-null  object
 3   app_tags   23183 non-null  object
 4   review     23183 non-null  object
 5   fps        23183 non-null  bool  
 6   voted_up   23183 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 951.0+ KB


In [45]:
final_df["voted_up"].value_counts(normalize=True)

True     0.505888
False    0.494112
Name: voted_up, dtype: float64

In [46]:
final_df["app_title"].value_counts(normalize=True)

Counter-Strike: Global Offensive    0.129405
Garry's Mod                         0.128888
Left 4 Dead 2                       0.073761
Arma 3                              0.050727
Portal 2                            0.049778
                                      ...   
Xotic                               0.000086
Hamilton's Great Adventure          0.000086
Penguins Arena: Sedna's World       0.000043
Pirates, Vikings, and Knights II    0.000043
Empires Mod                         0.000043
Name: app_title, Length: 166, dtype: float64

In [47]:
final_df.to_csv("../../../data/thanos.csv", index=False)